In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import *
import time


In [0]:
df = spark.read.format("parquet")\
    .load("abfss://bronze@ssdatalakeeteproject.dfs.core.windows.net/customers")

In [0]:
df.display()

In [0]:
df = df.drop("_resuced_data")

In [0]:
df.display()


In [0]:
df = df.withColumn("domains",split(col('email'),'@')[1])
df.display()

In [0]:
df.groupBy("domains").agg(count("customer_id").alias("total_customers")).sort("total_customers",ascending = False)\
.display()

In [0]:
df_gmail = df.filter(col("domains") == "gmail.com")
display(df_gmail)
time.sleep(5)

df_yahoo = df.filter(col("domains") == "yahoo.com")
display(df_yahoo)
time.sleep(5)

df_hotmail = df.filter(col("domains") == "hotmail.com")
display(df_hotmail)
time.sleep(5)



In [0]:
df = df.withColumn("Full_Name",concat(col('first_name'),lit(' '),col("last_name")))
df.display()
df = df.drop("first_name","last_name")
df.display()

In [0]:
#df.write.mode("append").format("delta").save("abfss://silver@ssdatalakeeteproject.dfs.core.windows.net/customers")

In [0]:
df = spark.read.format("delta").load("abfss://silver@ssdatalakeeteproject.dfs.core.windows.net/customers")

df.display()

In [0]:
df = df.groupBy("customer_id").count().filter("count > 1")\
.display(df)



In [0]:
df = spark.read.format("delta").load("abfss://silver@ssdatalakeeteproject.dfs.core.windows.net/customers")
df.display()


In [0]:
df = spark.read.format("delta").load("abfss://silver@ssdatalakeeteproject.dfs.core.windows.net/customers")
df = df.dropDuplicates(["customer_id"])
df.write.format("delta").mode("overwrite").save("abfss://silver@ssdatalakeeteproject.dfs.core.windows.net/customers")
df.display()

In [0]:
%sql
create schema if not exists databricks_catalog.silver

In [0]:
%sql 
create table if not exists databricks_catalog.silver.customer_silver
using delta
location "abfss://silver@ssdatalakeeteproject.dfs.core.windows.net/customers"


In [0]:
%sql
select * from databricks_catalog.silver.customer_silver


In [0]:
df = df.drop("_rescued_data")
df.write.format("delta").mode("overwrite").option("overwriteSchema", "true") \
  .saveAsTable("databricks_catalog.silver.customer_silver")

In [0]:
%sql
select * from databricks_catalog.silver.customer_silver
